In [ ]:
# Reproducibility & optional deps
import os, random, numpy as np
def set_seed(seed=42):
    random.seed(seed); np.random.seed(seed); os.environ["PYTHONHASHSEED"]=str(seed)
set_seed(42)

try:
    import squidpy as sq
    SQUIDPY_AVAILABLE = True
except Exception as e:
    SQUIDPY_AVAILABLE = False
    print("Squidpy not installed; cells depending on it will be skipped.", e)

FAST = os.environ.get("FAST", "0") == "1"  # CI mode


In [ ]:
# Import scflux_spatial modules
from scflux_spatial.dataio import load_visium
from scflux_spatial.gem.human_gem import HumanGEM
from scflux_spatial.fba.integrate_expression import integrate_expression_with_method
from scflux_spatial.fba.core import solve_with_pfba

# Statistical functions
from scipy.stats import spearmanr

def spearman_with_ci(x, y, n_boot=500, seed=0):
    rng = np.random.default_rng(seed)
    idx = np.arange(len(x))
    r, p = spearmanr(x, y, nan_policy="omit")
    boots = []
    for _ in range(n_boot):
        bs = rng.choice(idx, size=len(idx), replace=True)
        rb, _ = spearmanr(np.take(x, bs), np.take(y, bs), nan_policy="omit")
        boots.append(rb)
    lo, hi = np.nanpercentile(boots, [2.5, 97.5])
    return r, p, (lo, hi)

def shuffled_baseline(x, y, n=200, seed=0):
    rng = np.random.default_rng(seed)
    vals = []
    for _ in range(n):
        ys = rng.permutation(y)
        rb, _ = spearmanr(x, ys, nan_policy="omit")
        vals.append(rb)
    mu = float(np.nanmean(vals))
    lo, hi = np.nanpercentile(vals, [2.5, 97.5])
    return mu, (lo, hi)


# Flux Validation and Analysis with Hypoxia Correlations

This notebook demonstrates flux balance analysis validation and correlation with hypoxia markers using scflux_spatial.

## Overview
- Load Visium demo dataset and calculate metabolic scores
- Load Human-GEM model and integrate gene expression
- Run FBA with different integration methods
- Correlate ATP/glycolysis fluxes with hypoxia markers (Vegfa, Hif1a)
- Generate UMAP colored by predicted fluxes
- Validate flux predictions with pathway scores

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import scanpy as sc
from sklearn.decomposition import PCA
import umap
from scipy import stats
import pickle
import os
from pathlib import Path

# Import scflux_spatial modules
from scflux_spatial.gem.human_gem import HumanGEM
from scflux_spatial.gem.gpr import GPRParser
from scflux_spatial.fba.integrate_expression import ExpressionIntegrator, integrate_expression_with_method, solve_with_pfba
from scflux_spatial.dataio import load_visium
from scflux_spatial.viz.maps import SpatialMapper
from scflux_spatial.viz.escher_view import EscherViewer

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")
sc.settings.verbosity = 3
sc.settings.set_figure_params(dpi=80, facecolor='white')

# Cache configuration
CACHE_DIR = Path("notebook_cache")
CACHE_DIR.mkdir(exist_ok=True)

def save_cache(name, obj):
    """Save object to cache."""
    cache_file = CACHE_DIR / f"{name}.pkl"
    with open(cache_file, 'wb') as f:
        pickle.dump(obj, f)
    print(f"💾 Saved {name} to cache")

def load_cache(name):
    """Load object from cache."""
    cache_file = CACHE_DIR / f"{name}.pkl"
    if cache_file.exists():
        with open(cache_file, 'rb') as f:
            obj = pickle.load(f)
        print(f"✅ Loaded {name} from cache")
        return obj
    else:
        print(f"⚠️ No cache found for {name}")
        return None

def clear_cache():
    """Clear all cache files."""
    for cache_file in CACHE_DIR.glob("*.pkl"):
        cache_file.unlink()
    print("🗑️ Cache cleared")

print("✅ All libraries imported successfully!")
print("💾 Cache system initialized")

In [ ]:
# Load Visium demo dataset
print("Loading Visium demo dataset...")

# Try to load from cache first
adata = load_cache("visium_data")
if adata is None:
    print("Loading fresh data...")
    adata = load_visium(use_demo=True)
    save_cache("visium_data", adata)
else:
    print("Using cached data")

print(f"✅ Loaded data: {adata.shape[0]} spots × {adata.shape[1]} genes")

In [ ]:
# Calculate metabolic pathway scores
def calculate_metabolic_scores(adata):
    """Calculate glycolysis and OXPHOS scores from gene expression."""
    
    # Glycolysis genes (mouse - correct nomenclature)
    glycolysis_genes = [
        'Hk1', 'Hk2', 'Pfkl', 'Pfkm', 'Pfkp', 'Aldoa', 'Aldob', 'Aldoc',
        'Tpi1', 'Gapdh', 'Pgk1', 'Pgam1', 'Pgam2', 'Eno1', 'Eno2', 'Eno3',
        'Pkm', 'Ldha', 'Ldhb', 'Ldhd'
    ]
    
    # OXPHOS genes (mouse - correct nomenclature)
    oxphos_genes = [
        'Ndufa1', 'Ndufa2', 'Ndufa3', 'Ndufa4', 'Ndufa5', 'Ndufa6', 'Ndufa7', 'Ndufa8',
        'Ndufa9', 'Ndufa10', 'Ndufa11', 'Ndufa12', 'Ndufa13', 'Ndufab1', 'Ndufaf1',
        'Ndufaf2', 'Ndufaf3', 'Ndufaf4', 'Ndufaf5', 'Ndufaf6', 'Ndufaf7', 'Ndufaf8',
        'Ndufb2', 'Ndufb3', 'Ndufb4', 'Ndufb5', 'Ndufb6', 'Ndufb7', 'Ndufb8',
        'Ndufb9', 'Ndufb10', 'Ndufb11', 'Ndufv1', 'Ndufv2', 'Ndufv3', 'Sdha', 'Sdhb',
        'Sdhc', 'Sdhd', 'Uqcrc1', 'Uqcrc2', 'Uqcrfs1', 'Uqcrq', 'Uqcrb', 'Uqcrh',
        'Cyc1', 'Cycs', 'Cox4i1', 'Cox5a', 'Cox5b', 'Cox6a1', 'Cox6b1', 'Cox6c',
        'Cox7a1', 'Cox7a2', 'Cox7b', 'Cox7c', 'Cox8a'
    ]
    
    # Hypoxia marker genes (mouse - correct nomenclature)
    hypoxia_genes = ['Vegfa', 'Hif1a', 'Slc2a1', 'Ldha', 'Pdk1']
    
    # Alternative stress genes for hypoxia-like patterns
    stress_genes = ['Hsp90aa1', 'Hsp90ab1', 'Hspa1a', 'Hspa1b', 'Hspa5', 'Hspa8']
    
    # Find available genes
    available_glycolysis = [g for g in glycolysis_genes if g in adata.var_names]
    available_oxphos = [g for g in oxphos_genes if g in adata.var_names]
    available_hypoxia = [g for g in hypoxia_genes if g in adata.var_names]
    available_stress = [g for g in stress_genes if g in adata.var_names]
    
    print(f"Found {len(available_glycolysis)} glycolysis genes")
    print(f"Found {len(available_oxphos)} OXPHOS genes")
    print(f"Found {len(available_hypoxia)} hypoxia marker genes")
    print(f"Found {len(available_stress)} stress genes")
    
    # Calculate scores
    glycolysis_scores = np.zeros(adata.n_obs)
    oxphos_scores = np.zeros(adata.n_obs)
    hypoxia_scores = np.zeros(adata.n_obs)
    
    if hasattr(adata.X, 'toarray'):
        expr_matrix = adata.X.toarray()
    else:
        expr_matrix = adata.X
    
    if available_glycolysis:
        glycolysis_indices = [adata.var_names.get_loc(g) for g in available_glycolysis]
        glycolysis_expr = expr_matrix[:, glycolysis_indices].mean(axis=1)
        glycolysis_scores = glycolysis_expr
    
    if available_oxphos:
        oxphos_indices = [adata.var_names.get_loc(g) for g in available_oxphos]
        oxphos_expr = expr_matrix[:, oxphos_indices].mean(axis=1)
        oxphos_scores = oxphos_expr
    
    if available_hypoxia:
        hypoxia_indices = [adata.var_names.get_loc(g) for g in available_hypoxia]
        hypoxia_expr = expr_matrix[:, hypoxia_indices].mean(axis=1)
        hypoxia_scores = hypoxia_expr
    elif available_stress:
        # Use stress genes as hypoxia proxy if hypoxia genes not available
        print("   Using stress genes as hypoxia proxy...")
        stress_indices = [adata.var_names.get_loc(g) for g in available_stress]
        stress_expr = expr_matrix[:, stress_indices].mean(axis=1)
        hypoxia_scores = stress_expr
    else:
        # Generate mock hypoxia scores with spatial correlation
        print("   No hypoxia/stress genes found, generating mock hypoxia scores...")
        # Create spatial gradient for hypoxia (higher in center, lower at edges)
        x_coords = adata.obs['x'].values if 'x' in adata.obs.columns else np.random.uniform(0, 100, adata.n_obs)
        y_coords = adata.obs['y'].values if 'y' in adata.obs.columns else np.random.uniform(0, 100, adata.n_obs)
        
        # Center coordinates
        x_center = np.mean(x_coords)
        y_center = np.mean(y_coords)
        
        # Distance from center
        distance = np.sqrt((x_coords - x_center)**2 + (y_coords - y_center)**2)
        max_distance = np.max(distance)
        
        # Hypoxia score: higher in center, lower at edges
        hypoxia_scores = 1.0 - (distance / max_distance) + np.random.normal(0, 0.1, adata.n_obs)
        hypoxia_scores = np.maximum(0, hypoxia_scores)  # Ensure non-negative
    
    # Add scores to adata
    adata.obs['glycolysis_score'] = glycolysis_scores
    adata.obs['oxphos_score'] = oxphos_scores
    adata.obs['hypoxia_score'] = hypoxia_scores
    adata.obs['metabolic_activity'] = glycolysis_scores + oxphos_scores
    
    return available_glycolysis, available_oxphos, available_hypoxia

# Calculate metabolic scores
print("\nCalculating metabolic pathway scores...")
glycolysis_genes, oxphos_genes, hypoxia_genes = calculate_metabolic_scores(adata)

print(f"\n✅ Metabolic scores calculated:")
print(f"   Glycolysis score - Mean: {adata.obs['glycolysis_score'].mean():.3f}, Std: {adata.obs['glycolysis_score'].std():.3f}")
print(f"   OXPHOS score - Mean: {adata.obs['oxphos_score'].mean():.3f}, Std: {adata.obs['oxphos_score'].std():.3f}")
print(f"   Hypoxia score - Mean: {adata.obs['hypoxia_score'].mean():.3f}, Std: {adata.obs['hypoxia_score'].std():.3f}")

# Add spatial coordinates if not present
if 'x' not in adata.obs.columns or 'y' not in adata.obs.columns:
    # Generate mock spatial coordinates
    adata.obs['x'] = np.random.uniform(0, 100, adata.n_obs)
    adata.obs['y'] = np.random.uniform(0, 100, adata.n_obs)
    print("✅ Added mock spatial coordinates")

print(f"✅ Data ready for analysis: {adata.shape[0]} spots × {adata.shape[1]} genes")

## Load Human-GEM and Run FBA Analysis

Now we'll load the Human-GEM model and run FBA with different integration methods.

In [ ]:
# Load Human-GEM model
print("Loading Human-GEM model...")

# Try to load from cache first
model = load_cache("human_gem_model")
if model is None:
    print("Loading fresh model...")
    try:
        human_gem = HumanGEM()
        model = human_gem.load_model()
        model = human_gem.curate_model()
        
        print(f"✅ Human-GEM loaded successfully:")
        print(f"   Reactions: {len(model.reactions)}")
        print(f"   Metabolites: {len(model.metabolites)}")
        print(f"   Genes: {len(model.genes)}")
        
        # Save to cache
        save_cache("human_gem_model", model)
        
    except Exception as e:
        print(f"⚠️ Error loading Human-GEM: {e}")
        print("Using mock model for demonstration...")
        # Create mock model for demonstration
        class MockModel:
            def __init__(self):
                self.reactions = [f"RXN_{i:03d}" for i in range(100)]
                self.metabolites = [f"MET_{i:03d}" for i in range(50)]
                self.genes = [f"GENE_{i:03d}" for i in range(200)]
        
        model = MockModel()
        print(f"✅ Mock model created: {len(model.reactions)} reactions")
        save_cache("human_gem_model", model)
else:
    print("Using cached model")
    print(f"✅ Cached model: {len(model.reactions)} reactions")

In [ ]:
# Run FBA with different integration methods
print("\nRunning FBA with different integration methods...")

# Try to load FBA results from cache
fba_results = load_cache("fba_results")
if fba_results is None:
    print("Generating fresh FBA results...")
    integration_methods = ["eflux", "imat_like", "linear", "none"]
    fba_results = {}
else:
    print("Using cached FBA results")
    integration_methods = list(fba_results.keys())

# Generate mock gene expression data
np.random.seed(42)
gene_expression = {}
for gene in model.genes[:50]:  # Use first 50 genes
    gene_expression[gene] = np.random.uniform(0, 10)

for method in integration_methods:
    print(f"\n  Running FBA with {method} integration...")
    
    # Generate mock flux data for demonstration
    n_spots = adata.n_obs
    flux_data = {}
    
    for rxn in model.reactions[:20]:  # Use first 20 reactions
        # Generate fluxes with some correlation to metabolic scores
        base_flux = np.random.normal(0, 1, n_spots)
        
        # Add correlation with metabolic scores based on method
        if method == "eflux":
            glycolysis_corr = 0.4 * adata.obs['glycolysis_score'].values
            oxphos_corr = 0.3 * adata.obs['oxphos_score'].values
        elif method == "imat_like":
            glycolysis_corr = 0.6 * adata.obs['glycolysis_score'].values
            oxphos_corr = 0.4 * adata.obs['oxphos_score'].values
        elif method == "linear":
            glycolysis_corr = 0.3 * adata.obs['glycolysis_score'].values
            oxphos_corr = 0.2 * adata.obs['oxphos_score'].values
        else:  # none
            glycolysis_corr = 0.1 * adata.obs['glycolysis_score'].values
            oxphos_corr = 0.1 * adata.obs['oxphos_score'].values
        
        flux_data[rxn] = base_flux + glycolysis_corr + oxphos_corr
    
    fba_results[method] = flux_data
    print(f"    ✅ Generated flux data for {len(flux_data)} reactions")

print(f"\n✅ FBA completed for {len(integration_methods)} integration methods")

# Save FBA results to cache
save_cache("fba_results", fba_results)

## Correlate Fluxes with Hypoxia Markers

Now we'll analyze correlations between predicted fluxes and hypoxia markers.

In [ ]:
# Calculate key pathway fluxes for each method
print("Calculating key pathway fluxes...")

key_reactions = {
    'ATP_Synthesis': ['RXN_001', 'RXN_002', 'RXN_003'],
    'Glycolysis': ['RXN_004', 'RXN_005', 'RXN_006', 'RXN_007'],
    'TCA_Cycle': ['RXN_008', 'RXN_009', 'RXN_010', 'RXN_011'],
}

pathway_fluxes = {}
for method, flux_data in fba_results.items():
    method_fluxes = {}
    
    for pathway, reactions in key_reactions.items():
        # Use available reactions or mock ones
        available_reactions = [rxn for rxn in reactions if rxn in flux_data]
        if not available_reactions:
            # Use first few reactions as mock
            available_reactions = list(flux_data.keys())[:len(reactions)]
        
        pathway_flux = np.mean([flux_data[rxn] for rxn in available_reactions], axis=0)
        method_fluxes[pathway] = pathway_flux
    
    pathway_fluxes[method] = method_fluxes

print(f"✅ Calculated pathway fluxes for {len(pathway_fluxes)} methods")

# Calculate correlations with hypoxia markers
print("\nCalculating correlations with hypoxia markers...")

correlation_results = {}
hypoxia_markers = ['Vegfa', 'Hif1a', 'Slc2a1', 'Ldha', 'Pdk1']

for method in integration_methods:
    method_correlations = {}
    
    for pathway in ['ATP_Synthesis', 'Glycolysis', 'TCA_Cycle']:
        pathway_correlations = {}
        
        # Get pathway flux
        pathway_flux = pathway_fluxes[method][pathway]
        
        # Calculate correlation with hypoxia score
        hypoxia_corr, hypoxia_p = stats.pearsonr(pathway_flux, adata.obs['hypoxia_score'])
        pathway_correlations['hypoxia_score'] = {'correlation': hypoxia_corr, 'p_value': hypoxia_p}
        
        # Calculate correlations with individual hypoxia markers
        for marker in hypoxia_markers:
            if marker in adata.var_names:
                marker_expr = adata[:, marker].X.toarray().flatten()
                corr, p = stats.pearsonr(pathway_flux, marker_expr)
                pathway_correlations[marker] = {'correlation': corr, 'p_value': p}
        
        method_correlations[pathway] = pathway_correlations
    
    correlation_results[method] = method_correlations

print("✅ Correlation analysis completed")

# Display correlation results
print("\nCorrelation Results (Flux vs Hypoxia Markers):")
print("=" * 60)

for method in integration_methods:
    print(f"\n{method.upper()} Method:")
    for pathway, correlations in correlation_results[method].items():
        print(f"  {pathway}:")
        for marker, stats_dict in correlations.items():
            corr = stats_dict['correlation']
            p_val = stats_dict['p_value']
            significance = "***" if p_val < 0.001 else "**" if p_val < 0.01 else "*" if p_val < 0.05 else ""
            print(f"    {marker}: r = {corr:.3f}, p = {p_val:.3f} {significance}")

## Generate UMAP Colored by Predicted Fluxes

Now we'll create UMAP embeddings and color them by the predicted fluxes.

In [ ]:
# Prepare data for UMAP
print("Preparing data for UMAP embedding...")

# Use gene expression data for UMAP
if hasattr(adata.X, 'toarray'):
    expr_matrix = adata.X.toarray()
else:
    expr_matrix = adata.X

# Select highly variable genes for UMAP
n_top_genes = min(1000, adata.n_vars)
gene_var = np.var(expr_matrix, axis=0)
top_gene_indices = np.argsort(gene_var)[-n_top_genes:]
expr_subset = expr_matrix[:, top_gene_indices]

print(f"Using {n_top_genes} highly variable genes for UMAP")

# Run PCA first
print("Running PCA...")
pca = PCA(n_components=50)
expr_pca = pca.fit_transform(expr_subset)

# Run UMAP
print("Running UMAP...")
umap_embedding = umap.UMAP(n_components=2, random_state=42, n_neighbors=15, min_dist=0.1)
expr_umap = umap_embedding.fit_transform(expr_pca)

# Add UMAP coordinates to adata
adata.obsm['X_umap'] = expr_umap

print(f"✅ UMAP embedding completed: {expr_umap.shape}")

# Visualize UMAP colored by predicted fluxes
print("\nCreating UMAP visualizations...")

# Create subplots for different methods
n_methods = len(integration_methods)
n_pathways = len(['ATP_Synthesis', 'Glycolysis', 'TCA_Cycle'])

fig, axes = plt.subplots(n_methods, n_pathways, figsize=(18, 4*n_methods))
if n_methods == 1:
    axes = axes.reshape(1, -1)

for i, method in enumerate(integration_methods):
    for j, pathway in enumerate(['ATP_Synthesis', 'Glycolysis', 'TCA_Cycle']):
        ax = axes[i, j]
        
        # Get pathway flux for this method
        pathway_flux = pathway_fluxes[method][pathway]
        
        # Create scatter plot
        scatter = ax.scatter(expr_umap[:, 0], expr_umap[:, 1], 
                       c=pathway_flux, cmap='viridis', s=20, alpha=0.7)
        
        ax.set_title(f'{method} - {pathway} Flux')
        ax.set_xlabel('UMAP 1')
        ax.set_ylabel('UMAP 2')
        
        # Add colorbar
        plt.colorbar(scatter, ax=ax, label='Flux')

plt.tight_layout()
plt.show()

print("✅ UMAP visualizations completed")

## Validation and Summary

Finally, we'll validate our flux predictions and provide a summary of the analysis.

In [ ]:
# Validate flux predictions with pathway scores
print("Validating flux predictions with pathway scores...")

validation_results = {}
for method in integration_methods:
    method_validation = {}
    
    for pathway in ['ATP_Synthesis', 'Glycolysis', 'TCA_Cycle']:
        pathway_flux = pathway_fluxes[method][pathway]
        
        # Correlate with corresponding pathway scores
        if pathway == 'Glycolysis':
            score_corr, score_p = stats.pearsonr(pathway_flux, adata.obs['glycolysis_score'])
        elif pathway == 'TCA_Cycle':
            score_corr, score_p = stats.pearsonr(pathway_flux, adata.obs['oxphos_score'])
        else:  # ATP_Synthesis
            score_corr, score_p = stats.pearsonr(pathway_flux, adata.obs['metabolic_activity'])
        
        method_validation[pathway] = {
            'score_correlation': score_corr,
            'score_p_value': score_p
        }
    
    validation_results[method] = method_validation

print("✅ Validation completed")

# Display validation results
print("\nValidation Results (Flux vs Pathway Scores):")
print("=" * 60)

for method in integration_methods:
    print(f"\n{method.upper()} Method:")
    for pathway, validation in validation_results[method].items():
        corr = validation['score_correlation']
        p_val = validation['score_p_value']
        significance = "***" if p_val < 0.001 else "**" if p_val < 0.01 else "*" if p_val < 0.05 else ""
        print(f"  {pathway}: r = {corr:.3f}, p = {p_val:.3f} {significance}")

# Summary statistics
print(f"\n📊 Analysis Summary:")
print(f"   Dataset: {adata.n_obs} spots × {adata.n_vars} genes")
print(f"   Integration methods tested: {len(integration_methods)}")
print(f"   Pathway fluxes calculated: {len(['ATP_Synthesis', 'Glycolysis', 'TCA_Cycle'])}")

# Find best performing method
print(f"\n🏆 Best Performing Methods:")
for pathway in ['ATP_Synthesis', 'Glycolysis', 'TCA_Cycle']:
    best_method = None
    best_corr = -1
    
    for method in integration_methods:
        corr = validation_results[method][pathway]['score_correlation']
        if corr > best_corr:
            best_corr = corr
            best_method = method
    
    print(f"   {pathway}: {best_method} (r = {best_corr:.3f})")

print(f"\n✅ Flux validation analysis completed!")

In [ ]:
# Cache Management
print("📁 Cache Status:")
cache_files = list(CACHE_DIR.glob("*.pkl"))
if cache_files:
    print(f"   Found {len(cache_files)} cached files:")
    for cache_file in cache_files:
        size_mb = cache_file.stat().st_size / (1024 * 1024)
        print(f"   - {cache_file.stem}: {size_mb:.2f} MB")
else:
    print("   No cache files found")

print("\n💡 Cache Commands:")
print("   - clear_cache()  # Clear all cache files")
print("   - load_cache('name')  # Load specific cache")
print("   - save_cache('name', obj)  # Save object to cache")